Import Library

In [14]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
import itertools 

In this notebook, I want to analyze who is the best player in vct 2024 and how the best team will look like. At this point, I have no idea on what to do. But we will see.

In [2]:
player_data = pd.read_csv("dataset/vct_2024/players_stats/players_stats.csv")
player_data.head()

,Tournament,Stage,Match Type,Player,Teams,Agents,Rounds Played,Rating,Average Combat Score,Kills:Deaths,...,First Deaths Per Round,Headshot %,Clutch Success %,Clutches (won/played),Maximum Kills in a Single Map,Kills,Deaths,Assists,First Kills,First Deaths
0,Valorant Champions 2024,Playoffs,Upper Quarterfinals,Boo,Team Heretics,astra,26,0.75,138,0.58,...,0.08,21%,22%,2/9,11,11,19,14,1,2
1,Valorant Champions 2024,Playoffs,Upper Quarterfinals,Boo,Team Heretics,fade,16,1.10,185,1.38,...,0.00,21%,NaN,NaN,11,11,8,3,2,0
2,Valorant Champions 2024,Playoffs,Upper Quarterfinals,Boo,Team Heretics,"astra, fade",42,0.88,162,0.81,...,0.05,21%,22%,2/9,11,22,27,17,3,2
3,Valorant Champions 2024,Playoffs,Upper Quarterfinals,benjyfishy,Team Heretics,cypher,26,0.85,203,0.86,...,0.15,34%,NaN,NaN,18,18,21,3,2,4
4,Valorant Champions 2024,Playoffs,Upper Quarterfinals,benjyfishy,Team Heretics,killjoy,16,1.36,169,2.40,...,0.00,28%,50%,1/2,12,12,5,2,1,0


In [3]:
player_data.shape

(15030, 25)

In valorant, a team of 5 is usually consist of 1 duelist, 1 controller, 1 sentinel, 1 initiator, and 1 flex (can play most role if not all) player. Right now, I will not prioritize IGL within a team since I think it will make the code harder to do (and also because PRX can still play good without proper IGL). 

I will choose the best player for each role and combine them to make the best team. For someone to be considered as a duelist/sentinel/controller/initiator, they need to play the role for about 70 % of the time. For flex role, they should be able to play more than 10 agent to be considered as a flex. 

In [17]:
agent_dictionary = { 
    "sentinel": ["sage", "cypher", "killjoy", "chamber", "deadlock", "vyse"],
    "controller": ["brimstone", "viper", "omen", "astra", "harbor", "clove"],
    "duelist": ["jett", "reyna", "phoenix", "raze", "yoru", "neon", "iso"],
    "initiator": ["sova", "breach", "skye", "kayo", "fade", "gekko", "tejo"]
}

# flatten list
all_agents = list(itertools.chain.from_iterable([agent for agent in agent_dictionary.values()]))
all_agents

['sage',
 'cypher',
 'killjoy',
 'chamber',
 'deadlock',
 'vyse',
 'brimstone',
 'viper',
 'omen',
 'astra',
 'harbor',
 'clove',
 'jett',
 'reyna',
 'phoenix',
 'raze',
 'yoru',
 'neon',
 'iso',
 'sova',
 'breach',
 'skye',
 'kayo',
 'fade',
 'gekko',
 'tejo']

as you can see in the bottom, for one match, there are a "compiled" version of the game. For example, index 0 is game 1 and index 1 is game 2, meanwhile index 2 is the combined version of game 1 and game 2. For the sake of simplicity, I will just leave the "compiled" version and just use the single version. 

In [6]:
player_data.head(3)

,Tournament,Stage,Match Type,Player,Teams,Agents,Rounds Played,Rating,Average Combat Score,Kills:Deaths,...,First Deaths Per Round,Headshot %,Clutch Success %,Clutches (won/played),Maximum Kills in a Single Map,Kills,Deaths,Assists,First Kills,First Deaths
0,Valorant Champions 2024,Playoffs,Upper Quarterfinals,Boo,Team Heretics,astra,26,0.75,138,0.58,...,0.08,21%,22%,2/9,11,11,19,14,1,2
1,Valorant Champions 2024,Playoffs,Upper Quarterfinals,Boo,Team Heretics,fade,16,1.10,185,1.38,...,0.00,21%,NaN,NaN,11,11,8,3,2,0
2,Valorant Champions 2024,Playoffs,Upper Quarterfinals,Boo,Team Heretics,"astra, fade",42,0.88,162,0.81,...,0.05,21%,22%,2/9,11,22,27,17,3,2


In [20]:
player_data = player_data[~player_data["Agents"].str.contains(",")]

next, I want to check what are the agents played since there are new agents in 2025

In [21]:
for agent in all_agents:
    if agent not in player_data["Agents"].unique().tolist():
        print(f"this agent {agent} is not on the list")

this agent vyse is not on the list
this agent tejo is not on the list


vyse was released in august 2024 (after champions 2024) and tejo was released in january 2025. This shows that this two agent were not available back then in vct 2024 season.

In [ ]:
player_data["Player"].nunique() # there are 260 players

Next, since I'm determining who is the best player for each role in 2024, I think I will change the table a bit so that the player will be the main variable. From the code above, we can also know that there are 260 players in total.

I'm planning to group each player and get several important data from that and joined it all later in one dataframe like this. 

example : 
                
    player name      tournament played   regional winner     international winner    acs     kd      hs%       rating

    f0rsaken           20                   3                   0               500     3.4     50          1.76

    woot               100                  4                   0               400     2.4     24          1.4

    d4v4i              30                   5                   0               300     1.2     35          1.2

In [33]:
df = player_data.copy() 

data = df.groupby("Player")
data.get_group("Boo")

,Tournament,Stage,Match Type,Player,Teams,Agents,Rounds Played,Rating,Average Combat Score,Kills:Deaths,...,First Deaths Per Round,Headshot %,Clutch Success %,Clutches (won/played),Maximum Kills in a Single Map,Kills,Deaths,Assists,First Kills,First Deaths
0,Valorant Champions 2024,Playoffs,Upper Quarterfinals,Boo,Team Heretics,astra,26,0.75,138,0.58,...,0.08,21%,22%,2/9,11,11,19,14,1,2
1,Valorant Champions 2024,Playoffs,Upper Quarterfinals,Boo,Team Heretics,fade,16,1.10,185,1.38,...,0.00,21%,NaN,NaN,11,11,8,3,2,0
127,Valorant Champions 2024,Playoffs,Upper Semifinals,Boo,Team Heretics,fade,23,0.78,121,0.60,...,0.09,30%,0%,0/4,9,9,15,9,0,2
128,Valorant Champions 2024,Playoffs,Upper Semifinals,Boo,Team Heretics,viper,28,1.33,235,1.41,...,0.04,51%,29%,2/7,24,24,17,4,4,1
193,Valorant Champions 2024,Playoffs,Grand Final,Boo,Team Heretics,astra,22,0.46,131,0.56,...,0.00,22%,20%,1/5,10,10,18,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14449,Champions Tour 2024: EMEA Kickoff,Group Stage,Opening (A),Boo,Team Heretics,omen,24,1.49,297,1.53,...,0.04,35%,0%,0/2,26,26,17,10,1,1
14533,Champions Tour 2024: EMEA Kickoff,Group Stage,Winner's (A),Boo,Team Heretics,omen,34,1.34,203,1.87,...,0.03,29%,0%,0/4,18,28,15,12,5,1
14815,Champions Tour 2024: EMEA Kickoff,All Stages,All Match Types,Boo,Team Heretics,astra,23,1.70,299,2.17,...,0.09,45%,0%,0/2,26,26,12,7,4,2
14816,Champions Tour 2024: EMEA Kickoff,All Stages,All Match Types,Boo,Team Heretics,omen,148,1.04,185,1.01,...,0.06,28%,0%,0/17,26,100,99,56,10,9
